## Code for face detection

In [7]:
from ultralytics import YOLO
import cv2
import os
from PIL import Image
import numpy as np

In [2]:
%cd "D:\\Data\\Images\\Covid"

D:\Data\Images\Covid


In [3]:
%pwd

'D:\\Data\\Images\\Covid'

In [12]:
input_dir = "images_2"
output_dir_detected = "output_images_detected"  # For images with detected persons
output_dir_no_person = "output_images_no_person_2"  # For images without detected persons
os.makedirs(output_dir_detected, exist_ok=True)
os.makedirs(output_dir_no_person, exist_ok=True)

In [5]:
# Load a pre-trained YOLO model
# YOLOv8 model is used here; it can detect "person" as one of its classes
model = YOLO("yolov8n.pt")  # 'n' stands for Nano

In [ ]:
# Process each image in the input directory
for image_name in os.listdir(input_dir):
    image_path = os.path.join(input_dir, image_name)
    try:
        # Load the image
        image = cv2.imread(image_path)
        if image is None:
            print(f"Skipping invalid image: {image_name}")
            continue

        # Run object detection
        results = model(image)
        boxes = results[0].boxes

        # Check if boxes are detected
        if boxes is None or len(boxes) == 0:
            output_path_no_person = os.path.join(output_dir_no_person, image_name)
            Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).save(output_path_no_person)
            print(f"No objects detected, saved to no-person folder: {image_name}")
            continue

        # Extract detected classes
        detected_classes = []
        for box in boxes: 
            class_id = int(box.cls.item())
            detected_classes.append(model.names[class_id]) # Map ID to class name

        # Check if 'person' class is detected
        # detected_classes = [result['name'] for result in results[0].boxes.data.to('cpu').numpy()]
        if 'person' in detected_classes:
            # Save the image to the detected folder
            output_path_detected = os.path.join(output_dir_detected, image_name)
            Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).save(output_path_detected)
            print(f"Person detected, saved: {image_name}")
        else:
            # Save the image to the no-person folder
            output_path_no_person = os.path.join(output_dir_no_person, image_name)
            Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)).save(output_path_no_person)
            print(f"No person detected, saved to no-person folder: {image_name}")

    except Exception as e:
        print(f"Error processing {image_name}: {e}")

print("Dataset cleaning complete.")